In [1]:
# Combine_zips_with_any_intersection
import pandas as pd
import numpy as np
import datetime
import json
from haversine import haversine
import zipcodes
import googlemaps
import os
import zipcodes
os.getcwd()

'/home/jian/Projects/Smoothie_King/TA/New_TA_20190108'

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [3]:
# 0930 closed as in the email "SK stores missing sales data"
store_list=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/StoreList_20190107.xlsx",dtype=str)
store_list=store_list[~pd.isnull(store_list['storenumber'])]
closed_stores=store_list[store_list['status']=="Closed"]
store_list=store_list[store_list['status']!="Closed"]
store_list=store_list[store_list['storenumber']!="0930"]
closed_stores=closed_stores.append(store_list['storenumber']=="0930")
store_list=store_list[store_list['storenumber']!="nan"].reset_index()
del store_list['index']

store_list['Latitude']=store_list['Latitude'].astype(float)
store_list['Longitude']=store_list['Longitude'].astype(float)
store_list['zip']=store_list['zip'].apply(lambda x: x.split("-")[0])

store_list.shape

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:4528: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  other.index).difference(self.columns).tolist()


(905, 14)

In [4]:
non_lat_lng_store=store_list[pd.isnull(store_list['Latitude']) | pd.isnull(store_list['Longitude'])]
store_list_with_latlng=store_list[~pd.isnull(store_list['Latitude']) & ~pd.isnull(store_list['Longitude'])]

In [5]:
# Google "10560 Westheimer Road, Houston, TX, 77042"
# Get 29.737266,-95.5651062
non_lat_lng_store['Latitude']=29.737266
non_lat_lng_store['Longitude']=-95.5651062

store_list=non_lat_lng_store.append(store_list_with_latlng).reset_index()
store_list=store_list.sort_values(['index'])
del store_list['index']
store_list=store_list.drop_duplicates() #Remove dup row 0443

store_list=store_list.reset_index()
del store_list['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
# Manually update the zip for the 11 stores
store_zip_dic_updated={}
store_zip_dic_updated.update({"1333":"36695"})
store_zip_dic_updated.update({"1349":"07747"})
store_zip_dic_updated.update({"1436":"78411"})

store_zip_dic_updated.update({"1245":"68133"})
store_zip_dic_updated.update({"1152":"70810"})
store_zip_dic_updated.update({"1282":"70803"})

store_zip_dic_updated.update({"1386":"62711"})
store_zip_dic_updated.update({"1467":"63146"})
store_zip_dic_updated.update({"1252":"73013"})

store_zip_dic_updated.update({"1397":"28054"})
store_zip_dic_updated.update({"1653":"46074"})

store_zip_dic_updated.update({"1390":"39440"}) # Changed
store_zip_dic_updated.update({"1464":"07901"}) # Changed

store_zip_dic_updated.update({"0989":"70130"}) # Changed
store_zip_dic_updated.update({"1457":"08873"}) # Changed
store_zip_dic_updated.update({"1432":"30281"}) # Changed




In [7]:
store_without_zips=store_list[store_list['storenumber'].isin(store_zip_dic_updated.keys())]
store_with_zips=store_list[~store_list['storenumber'].isin(store_zip_dic_updated.keys())]
store_without_zips['zip']=store_without_zips['storenumber'].apply(lambda x: store_zip_dic_updated[x])

store_list=store_with_zips.append(store_without_zips).reset_index()
store_list=store_list.sort_values(['index'])
del store_list['index']
store_list=store_list.reset_index()
del store_list['index']

store_zip5_dic=store_list.set_index(['storenumber'])['zip'].to_dict()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
check_store_zip_to_store_loc=store_list.copy()
check_store_zip_to_store_loc=check_store_zip_to_store_loc.reset_index()
check_store_zip_to_store_loc['orignal_dist']=np.nan
del check_store_zip_to_store_loc['index']
for i in range(len(check_store_zip_to_store_loc)):
    store_zip=check_store_zip_to_store_loc['zip'][i]
    store_lat=check_store_zip_to_store_loc['Latitude'][i]
    store_lng=check_store_zip_to_store_loc['Longitude'][i]
    if store_zip in zip_centers.keys():
        dist_org=haversine(zip_centers[store_zip],[store_lat,store_lng],miles=True)
    else:
        dist_org=np.nan
    check_store_zip_to_store_loc['orignal_dist'][i]=dist_org
    
check_store_zip_to_store_loc=check_store_zip_to_store_loc.sort_values(['orignal_dist'],ascending=False)
check_store_zip_to_store_loc_revise=check_store_zip_to_store_loc[check_store_zip_to_store_loc['orignal_dist']>=20]
check_store_zip_to_store_loc_revise.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


(12, 15)

In [9]:
# Manually google the 14 address for lat lng
to_update_lat_lng_dict={}
to_update_lat_lng_dict.update({"1385":[32.552785,-83.6603137]}) #770 Hwy 96, Bonaire, GA, 31005
to_update_lat_lng_dict.update({"1149":[30.0818157,-95.5229529]}) #21630 Kuykendahl Rd. Suite 405, Spring, TX, 77388
to_update_lat_lng_dict.update({"1587":[35.9632665,-78.9601981]}) #4215 University Dr, Suite 100, Durham, NC, 27707
to_update_lat_lng_dict.update({"1382":[34.0897496,-81.1726416]}) #945-G Lake Murray Blvd, Irmo, SC, 29063
# to_update_lat_lng_dict.update({"1171":[32.2832495,-90.0270132]}) #201 Woodgate Drive S, Brandon, MS, 39402, Correct
to_update_lat_lng_dict.update({"0262NT":[29.7181022,-95.3408497]}) #University of Houston, Houston, TX, 77004  ########## 4500 University Dr, Houston, TX 77004

to_update_lat_lng_dict.update({"1363":[35.7489926,-81.336297]}) #225 12th Ave. NE, Hickory, NC, 28601
# to_update_lat_lng_dict.update({"1358":[35.8363716,-78.6454606]}) #4120 Main at North Hills St. Suite 110, Raleigh, NC, 27403, Correct
to_update_lat_lng_dict.update({"1464":[40.717702,-74.3630287]}) #462 Springfield Ave, Summit, NJ, 08053
# to_update_lat_lng_dict.update({"1673":[34.7840431,-86.9484213]}) #1260 US Highway 72 E, Suite 1, Athens, AL, 35661, Correct
# to_update_lat_lng_dict.update({"1303":[34.0211345,-84.3198411]}) #1530 Old Alabama Road, Suite 180, Roswell, GA, 30094, Correct
to_update_lat_lng_dict.update({"1190":[33.8056582,-84.5279771]}) #1025 Veterans Memorial Hwy SE, Bldg 710, Mableton, GA, 30126
to_update_lat_lng_dict.update({"0116":[29.8767939,-95.6450357]}) #6810 Hwy 6N, Houston, TX, 77084

In [10]:
store_list_keep=store_list[~store_list['storenumber'].isin(to_update_lat_lng_dict.keys())]
store_list_revised=store_list[store_list['storenumber'].isin(to_update_lat_lng_dict.keys())]
store_list_revised['Latitude']=store_list_revised['storenumber'].apply(lambda x: to_update_lat_lng_dict[x][0])
store_list_revised['Longitude']=store_list_revised['storenumber'].apply(lambda x: to_update_lat_lng_dict[x][1])

store_list=store_list_keep.append(store_list_revised).reset_index()
store_list=store_list.sort_values(['index'])
del store_list['index']
store_list=store_list.reset_index()
del store_list['index']

store_lat_dic=store_list.set_index(['storenumber'])['Latitude'].to_dict()
store_lng_dic=store_list.set_index(['storenumber'])['Longitude'].to_dict()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
store_zips_3_miles_long=pd.DataFrame()
store_zips_3_miles_dic={}
for store_number in store_list['storenumber'].tolist():
    store_center=[store_lat_dic[store_number],store_lng_dic[store_number]]
    store_zip_list=[store_zip5_dic[store_number]]
    df=pd.DataFrame({"store_number":store_number,"zip_code":store_zip_list,"distance":"store_zip"},index=[0])
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=3:
            store_zip_list=store_zip_list+[zip_cd]
            df=df.append(pd.DataFrame({"store_number":store_number,"zip_code":zip_cd,"distance":dist},index=[0]))
    
    store_zips_3_miles_long=store_zips_3_miles_long.append(df)
    store_zip_list=list(set(store_zip_list))
    store_zips_3_miles_dic.update({store_number:store_zip_list})
store_list['zip_cd_list']=store_list['storenumber'].apply(lambda x: store_zips_3_miles_dic[x])

In [12]:
store_list_changed=store_list[(store_list['storenumber'].isin(store_zip_dic_updated.keys())) | (store_list['storenumber'].isin(to_update_lat_lng_dict.keys()))]
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/revised_addresses_info_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
store_list_changed.to_excel(writer,"store_list_changed",index=False)
writer.save()

In [13]:
store_zips_3_miles_long.head(2)

,distance,store_number,zip_code
0,store_zip,1128,75219
0,2.09968,1128,75250


In [14]:
store_list.shape

(904, 15)

In [15]:
store_list.columns.tolist()

['storenumber',
 'Address',
 'city',
 'state',
 'zip',
 'StoreOpenDate',
 'Latitude',
 'Longitude',
 'Ownership',
 'Franchisee_Name',
 'IPAddress',
 'DriveThru',
 'status',
 'StoreCloseDate',
 'zip_cd_list']

# Create TA from Zip list

In [16]:
data=store_list[['storenumber','Address','city','state','zip','StoreOpenDate','Latitude','Longitude','Ownership',
           'Franchisee_Name','IPAddress','DriveThru','status','StoreCloseDate','zip_cd_list']]
data=data.sort_values('storenumber')
data=data.rename(columns={"zip_cd_list":"zips_3_miles","zip":"revised_store_zip"})

In [17]:
data['TA']=np.nan
data['TA']=1
data=data.reset_index()
del data['index']
df_TA_zips=pd.DataFrame({"store":[data['storenumber'][0]]*len(data['zips_3_miles'][0]),"zip":data['zips_3_miles'][0],"TA":[1]*len(data['zips_3_miles'][0])},index=[1]*len(data['zips_3_miles'][0]))


In [18]:
TA_counter=1

for i in range(1,len(data)):
    intersection_zip=list(set(data['zips_3_miles'][i]).intersection(set(df_TA_zips['zip'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),"zip":data['zips_3_miles'][i],"TA":[TA_counter]*len(data['zips_3_miles'][i])},index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),"zip":data['zips_3_miles'][i],"TA":[selected_TA]*len(data['zips_3_miles'][i])},index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
dict_TA_zips=df_TA_zips.set_index('zip').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('store').to_dict()['TA']
data['TA']=data['storenumber'].apply(lambda x: dict_TA_store[x])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [19]:
data.shape

(904, 16)

In [20]:
data['TA'].max()

536

In [21]:
df_ta_num_unique=data[['TA']].drop_duplicates().reset_index()
del df_ta_num_unique['index']
df_ta_num_unique['new_TA']=[x+1 for x in range(len(df_ta_num_unique))]

dict_ta_num_unique=df_ta_num_unique.set_index(['TA']).to_dict()['new_TA']
data['TA']=data['TA'].apply(lambda x: dict_ta_num_unique[x])
df_TA_zips['TA']=df_TA_zips['TA'].apply(lambda x: dict_ta_num_unique[x])


In [22]:

summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
# summary_store_2=df_TA_zips[['TA','store']].drop_duplicates().groupby("TA")['store'].count().to_frame().reset_index().rename(columns={"store":"store_count_2"})
summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})



summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

# summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"revised_store_zip":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


In [23]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

In [24]:
df_TA_zips['zip'].apply(lambda x: type(x)).unique()

array([<class 'str'>], dtype=object)

In [25]:
df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_zip_dist_in_TA['index']

output_distance_zip_in_TA=pd.DataFrame()
counter_k=0
zip_not_found_list=[]


for ta,group in df_zip_dist_in_TA.groupby(['TA']):
    group=group.reset_index()
    del group['index']
    
    if len(group)>1:
    
        dist_list=[]

        for i in range(len(group)):
            zip_hold=group['zip'][i]

            if zip_hold not in zip_centers.keys():
                try:
                    zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                except:
                    print("zip not found hold, ",zip_hold)
                    zip_not_found_list=zip_not_found_list+[zip_hold]

            else:
                zip_hold_center=zip_centers[zip_hold]

            for j in range(i+1,len(group)):
                zip_var=group['zip'][j]
                if zip_var not in zip_centers.keys():
                    try:
                        zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                    except:
                        print("zip not found var, ",zip_hold)
                        zip_not_found_list=zip_not_found_list+[zip_var]

                else:
                    zip_var_center=zip_centers[zip_var]

                try:
                    dist=haversine(zip_hold_center,zip_var_center,miles=True)
                except:
                    dist=np.nan

                dist_list=dist_list+[dist]
        df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
        counter_k+=1
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    else:
        df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    

In [26]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip','DMA']

df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=zipcodes.matching(x)[0]['city']
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=zipcodes.matching(x)[0]['state']
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"

df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_DMA_in_TA['index']
df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")
df_DMA_in_TA.head(2)



,TA,zip,DMA
0,3,70087,NEW ORLEANS
1,3,70031,NEW ORLEANS


In [27]:

df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})

In [28]:
manually_city_dict_of_zip={"29486":"Summerville"}
manually_state_dict_of_zip={"29486":"SC"}
df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
# zip "08644","76190" removed to determine cities/states because of not existing as in https://www.unitedstateszipcodes.org/ 

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [29]:
counter_k=1
df_primary_city_state=pd.DataFrame()
df_primary_DMA=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['TA']):
    df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

    
for ta,group in df_DMA_in_TA.groupby(['TA']):
    df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_DMA['index']
    primary_DMA=df_DMA['DMA'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
    counter_k+=1
    df_primary_DMA=df_primary_DMA.append(df)

In [30]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")

In [31]:
data['Latitude']=data['Latitude'].astype(float)
data['Longitude']=data['Longitude'].astype(float)
data['TA']=data['TA'].astype(str)
summary_by_TA['TA']=summary_by_TA['TA'].astype(str)

# Revise TV

In [32]:
summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                      (summary_by_TA['store_count']>=2) &\
                                     (summary_by_TA['dist_max']>12)]
summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                      (summary_by_TA['store_count']<2) |\
                                     (summary_by_TA['dist_max']<=12)]
summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
del summary_by_TA_to_revise['index']

summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
del summary_by_TA_to_keep['index']

In [33]:
len(summary_by_TA['TA'].unique())

495

In [34]:
summary_by_TA['TA'].apply(lambda x: type(x)).unique()

array([<class 'str'>], dtype=object)

In [35]:
summary_by_TA['TA'].astype(int).max()

495

In [36]:
loop_counter=0
while len(summary_by_TA_to_revise)>0:
    loop_counter+=1
    if loop_counter>30:
        break
    
    summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
    del summary_by_TA_to_revise['index']

    summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
    del summary_by_TA_to_keep['index']

    data_keep=data[data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data[~data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data_revise.reset_index()
    del data_revise['index']


    store_lat_dict=data_revise.set_index(['storenumber']).to_dict()['Latitude']
    store_lng_dict=data_revise.set_index(['storenumber']).to_dict()['Longitude']


    store_sub_df=pd.DataFrame()
    for i in range(len(summary_by_TA_to_revise)):
        TA=summary_by_TA_to_revise['TA'][i]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        initial_dist=0
        store_pair=[np.nan,np.nan]
        while len(store_list)>=2:
            store_hold=store_list[0]
            store_list.remove(store_hold)
            store_hold_center=[store_lat_dict[store_hold],store_lng_dict[store_hold]]
            for store_var in store_list:
                store_var_center=[store_lat_dict[store_var],store_lng_dict[store_var]]
                dist=haversine(store_hold_center,store_var_center,miles=True)
                if dist>initial_dist:
                    initial_dist=dist
                    store_pair=[store_hold,store_var]
        store_a=store_pair[0]
        store_b=store_pair[1]

        store_a_center=[store_lat_dict[store_a],store_lng_dict[store_a]]
        store_b_center=[store_lat_dict[store_b],store_lng_dict[store_b]]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        for store in store_list:
            store_center=[store_lat_dict[store],store_lng_dict[store]]
            dist_a=haversine(store_a_center,store_center,miles=True)
            dist_b=haversine(store_b_center,store_center,miles=True)
            if dist_a<dist_b:
                sub_group="a"
            else:
                sub_group="b"
            df=pd.DataFrame({"storenumber":store,"TA":TA,"sub_group":sub_group},index=[store])
            store_sub_df=store_sub_df.append(df)
    
    store_sub_df['TA']=store_sub_df['TA'].astype(str)
    store_sub_df['TA']=store_sub_df['TA']+"_"+store_sub_df['sub_group']
    store_sub_df=store_sub_df[['storenumber','TA']]
    store_sub_df_dic=store_sub_df.set_index(['storenumber']).to_dict()['TA']
                    
    data_revise['TA']=data_revise['storenumber'].apply(lambda x: store_sub_df_dic[x])
    data=data_keep.append(data_revise)
    data=data.sort_values(['storenumber'])
    data=data.reset_index()
    del data['index']
    data['TA']=data['TA'].apply(lambda x: str(x).zfill(5))

    df_TA_zips=pd.DataFrame()
    for i in range(len(data)):
        df=pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),
                         "TA":[data['TA'][i]]*len(data['zips_3_miles'][i]),
                         "zip":data['zips_3_miles'][i]},index=data['zips_3_miles'][i])
        df_TA_zips=df_TA_zips.append(df)


    summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
    summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
    summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
    summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})

    summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

    # summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
    TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"zip_code":"store_zip_list"})
    summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


    summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
    summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
    summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

    df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_zip_dist_in_TA['index']

    output_distance_zip_in_TA=pd.DataFrame()
    counter_k=0
    for ta,group in df_zip_dist_in_TA.groupby(['TA']):
        group=group.reset_index()
        del group['index']

        if len(group)>1:

            dist_list=[]

            for i in range(len(group)):
                zip_hold=group['zip'][i]

                if zip_hold not in zip_centers.keys():
                    try:
                        zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_hold_center=zip_centers[zip_hold]

                for j in range(i+1,len(group)):
                    zip_var=group['zip'][j]
                    if zip_var not in zip_centers.keys():
                        try:
                            zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                        except:
                            print("zip not found, ",zip_hold)

                    else:
                        zip_var_center=zip_centers[zip_var]

                    try:
                        dist=haversine(zip_hold_center,zip_var_center,miles=True)
                    except:
                        dist=np.nan

                    dist_list=dist_list+[dist]
            df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
            counter_k+=1
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
        else:
            df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)

    zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
    zip_DMA=zip_DMA.iloc[:,[0,2]]
    zip_DMA.columns=['zip','DMA']

    df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_city_state_in_TA['index']

    def city_of_zip(x):
        try:
            city=zipcodes.matching(x)[0]['city']
        except:
            city=np.nan
        return city

    def state_of_zip(x):
        try:
            state=zipcodes.matching(x)[0]['state']
        except:
            state=np.nan
        return state

    df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
    df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
    
    manually_city_dict_of_zip={"29486":"Summerville"}
    manually_state_dict_of_zip={"29486":"SC"}
    df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
    df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

    removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
    df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
    
    df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"

    df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_DMA_in_TA['index']
    df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")

    df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
    df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
    df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
    df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
    df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
    df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})

    counter_k
    df_primary_city_state=pd.DataFrame()
    df_primary_DMA=pd.DataFrame()
    
    df_city_state_in_TA
    for ta,group in df_city_state_in_TA.groupby(['TA']):
        df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_city['index']
        primary_city=df_city['city'][0]

        df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_state['index']
        primary_state=df_state['state'][0]

        df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
        counter_k+=1
        df_primary_city_state=df_primary_city_state.append(df)


    for ta,group in df_DMA_in_TA.groupby(['TA']):
        df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_DMA['index']
        primary_DMA=df_DMA['DMA'][0]

        df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
        counter_k+=1
        df_primary_DMA=df_primary_DMA.append(df)

    summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")
    summary_by_TA['Initial_TA']=summary_by_TA['TA'].apply(lambda x: int(x.split("_")[0]))
    summary_by_TA=summary_by_TA.sort_values(['Initial_TA','TA'])

    summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

    summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                          (summary_by_TA['store_count']>=2) &\
                                         (summary_by_TA['dist_max']>12)]
    summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                          (summary_by_TA['store_count']<2) |\
                                         (summary_by_TA['dist_max']<=12)]

In [37]:
summary_by_TA_output=pd.DataFrame()
summary_by_TA=summary_by_TA.sort_values(['Initial_TA'])
for old_ta,group in summary_by_TA.groupby(['Initial_TA']):
    if len(group)>1:
        group['TA']=[str(old_ta).zfill(3)+"_"+str(x+1) for x in range(len(group))]
    else:
        group['TA']=str(old_ta).zfill(3)
    summary_by_TA_output=summary_by_TA_output.append(group)
ta_store_list_dic=summary_by_TA_output.set_index(['TA']).to_dict()['store_list']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [38]:
store_to_new_ta_dict={}
for new_ta in ta_store_list_dic.keys():
    length=len(ta_store_list_dic[new_ta])
    df=pd.DataFrame({"store":ta_store_list_dic[new_ta],"new_ta":[new_ta]*length},index=[x for x in range(length)])
    df_dict=df.set_index(['store']).to_dict()['new_ta']
    store_to_new_ta_dict.update(df_dict)
len(store_to_new_ta_dict)    
data['TA']=data['storenumber'].apply(lambda x: store_to_new_ta_dict[x])
df_TA_zips['TA']=df_TA_zips['store'].apply(lambda x: store_to_new_ta_dict[x])

summary_by_store_count=summary_by_TA_output.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA_output.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

In [39]:
data_for_tableau=data[['storenumber','revised_store_zip','city','state','Latitude','Longitude','TA']]
for col in data_for_tableau.columns.tolist()[1:6]:
    data_for_tableau=data_for_tableau.rename(columns={col:"store_"+col})
data_for_tableau=data_for_tableau.append(df_TA_zips[['TA','zip']])
data_for_tableau['store_Latitude']=data_for_tableau['store_Latitude'].fillna(0)
data_for_tableau['store_Longitude']=data_for_tableau['store_Longitude'].fillna(0)

In [40]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary_by_TA_output.to_excel(writer,"summary_by_TA",index=False)
data['storenumber']=data['storenumber'].replace("0262NT","0262")
data['storenumber']=data['storenumber'].astype(int)
data=data.sort_values(['storenumber'])
data['storenumber']=data['storenumber'].astype(str)
data['storenumber']=data['storenumber'].apply(lambda x: x.zfill(4))
data['storenumber']=data['storenumber'].replace("0262","0262NT")

data.to_excel(writer,"output_TA_by_store",index=False)
df_TA_zips.to_excel(writer,"zip_TA",index=False)
data_for_tableau.to_excel(writer,"data_for_tableau",index=False)
summary_by_store_count.to_excel(writer,"summary_by_store_count",index=False)
closed_stores.to_excel(writer,"closed_stores",index=False)
overwrite_zip_stores=store_list_changed.copy()
overwrite_zip_stores.to_excel(writer,"stores_info_changed",index=False)
writer.save()